# Tensorflow Poison Frogs Fruits360 demo for Securing AI Lab deployment

## Setup

**Note:** This demo is specifically for the NCCoE DGX Workstation with hostname `dgx-station-2`.

Port forwarding is required in order to run this demo.
The recommended port mapping is as follows:

- Map `localhost:30080` on laptop to `localhost:30080` on `dgx-station-2`
- Map `localhost:35000` on laptop to `localhost:35000` on `dgx-station-2`

A sample SSH config file that enables the above port forwarding is provided below,

> ⚠️ **Edits required**: replace `username` with your assigned username _on the NCCoE virtual machines_!

```conf
# vm hostname: jumphost001
Host nccoe-jumphost001
    Hostname 10.33.53.98
    User username
    Port 54131
    IdentityFile %d/.ssh/nccoe-vm

# vm hostname: dgx-station-2
Host nccoe-k8s-gpu002
    Hostname 192.168.1.28
    User username
    Port 22
    IdentityFile %d/.ssh/nccoe-vm
    ProxyJump nccoe-jumphost001
    LocalForward 30080 localhost:30080
    LocalForward 35000 localhost:35000
```

Now, connect to the NCCoE VPN and SSH into the DGX Workstation,

```bash
ssh nccoe-k8s-gpu002
```

Next, we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [ ]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple
import tarfile
# Please enter custom username here.
USERNAME = "howard"

# Filter out warning messages
warnings.filterwarnings("ignore")

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:30080"

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_fruits360_poison_frogs"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

Check that the Makefile works in your environment by executing the `bash` code block below,

In [ ]:
os.getenv("AWS_ACCESS_KEY_ID")

In [ ]:
%%bash

# Running this will just list the available rules defined in the demo's Makefile.
make

## Submit and run jobs

The jobs that we will be running are implemented in the Python source files under `src/`, which will be executed using the entrypoints defined in the `MLproject` file.
To get this information into the architecture, we need to package those files up into an archive and upload it to the lab API.
For convenience, the `Makefile` provides a rule for creating the archive file, just run `make workflows`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the lab's RESTful API using the HTTP protocol.
We connect using the client below,

In [ ]:
restapi_client = utils.SecuringAIClient(address=RESTAPI_API_BASE)

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment named `"jglasbrenner_mnist"` exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

Next, we need to train our model.
We will be using the V100 GPUs that are available on the DGX Workstation, which we can use by submitting our job to the `"tensorflow_gpu"` queue.
We will train three models, a shallow network model, a LeNet-5 model, and an AlexNet model,

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_vgg16_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=20",
        "-P register_model=True",
        "-P model_architecture=vgg16",
        "-P epochs=30",
        "-P data_dir_train=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training",
        "-P data_dir_test=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Test",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Training job for VGG16 neural network submitted")
print("")
pprint.pprint(response_vgg16_train)

while mlflow_run_id_is_not_known(response_vgg16_train):
    time.sleep(1)
    response_vgg16_train = restapi_client.get_job_by_id(
        response_vgg16_train["jobId"]
    )


# Poison Frogs: Image Selection
### Now we will examine the test set and select a particular image we'd like to corrupt:

In [ ]:
# Model download:
import requests
import os
import tensorflow as tf



def load_model_from_job_run(run_id, model_dir):
    params = {"path":"model/data/model.h5", "run_uuid": run_id}
    r = requests.get('http://localhost:35000/get-artifact', params=params)
    print("Accessing MLflow Client Server. Downloading model files.")
    print(r.headers)
    model_path = os.path.join(model_dir, 'model.h5')
    
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        
    with open(model_path, "wb+") as f:
        f.write(r.content)
    
    return tf.keras.models.load_model(model_path)

def load_images_from_job_run(run_id, image_dir):
    params = {"path":"adversarial_poison.tar.gz", "run_uuid": run_id}
    r = requests.get('http://localhost:35000/get-artifact', params=params)
    print("Accessing MLflow Client Server. Downloading image files.")
    print(r.headers)
    zip_path = os.path.join(image_dir, 'adversarial_poison.tar.gz')
    
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
        
    with open(zip_path, "wb+") as f:
        f.write(r.content)
    
    with tarfile.open(zip_path, "r:gz") as f:
        f.extractall(path=image_dir)
    
# Wait for model to become available.
while response_vgg16_train['status'] != "finished":
    time.sleep(1)
    response_vgg16_train = restapi_client.get_job_by_id(
        response_vgg16_train["jobId"]
    )  
    
original_model = load_model_from_job_run(response_vgg16_train['mlflowRunId'], './original_model')
        

In [ ]:
from os import listdir
from os.path import isfile, join
import random


def create_class_list(imagedir):
    class_names = []
    for item in os.listdir(imagedir):
        if os.path.isdir(imagedir+"/"+item):
            class_names.append(item)
    return sorted(class_names)

class_names = create_class_list("/nfs/1/datasets/Fruits360-Kaggle-2019/fruits-360/Test")

image_list = []
for path, subdirs, files in os.walk("/nfs/1/datasets/Fruits360-Kaggle-2019/fruits-360/Test"):
    for name in files:
        image_list.append(os.path.join(path, name))
        
class_names

Here we'll randomly parse through images of interest. 

The code below will pull an image from the MNIST test set, and run it through the trained mode.

For this demo, we'd like to start with an image that the model predicts correctly.

Rerun the the cell below until an image of interest is selected. Then proceed to the rest of the demo.

In [ ]:
random_file = np.random.choice(image_list)
(label,filename) = random_file.split('/')[-2:]

# load and show an image with Pillow
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt

# Open the image form working directory
image = Image.open(random_file)
image = image.resize((224, 224))

# Summarize some details about the image
print("Image label: " + str(label) )
predicted_label = class_names[np.argmax(original_model.predict(np.reshape(image, (1,224,224,3))), axis=1)[0]]
print("Predicted label: " + predicted_label)
print("Filename: " + filename)

# Show the image
image = np.asarray(image)
fig = plt.figure
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
#Image label: Redcurrant
#Predicted label: Redcurrant
#Filename: r_248_100.jpg

# Set target image to propogate into clean label images from another class
target_file = filename #"r_5_100.jpg"
original_label = label #"Passion Fruit"

# Set target label here, this will be the new label that the target image will be reclassified as.
target_label = class_names[0]

### Generating and Deploying Poisoned Images.

Now we will create our set of poisoned images and examine them visually.

Start by submitting the poison generation job below.

In [ ]:
# Create poisoned images.
fixed_original_label = original_label.replace(" ", "\\ ")
response_gen_poison_vgg16 = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_poison",
    entry_point_kwargs=" ".join(
        [
            f"-P model={EXPERIMENT_NAME}_vgg16/1",
            "-P model_architecture=vgg16",
            "-P data_dir=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training",
            f"-P target_image_path=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Test/{fixed_original_label}/{target_file}",
            f"-P target_class={class_names.index(target_label)}",
            "-P feature_layer_index=-2",
            "-P max_iter=100",
            "-P similarity_coeff=512",
            "-P watermark=.3",
            "-P batch_size=100",
            "-P num_poisoned_batches=1"
            
        ]
    ),
    queue="tensorflow_cpu",
    depends_on=response_vgg16_train["jobId"],
)

print("Poison frogs attack (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_gen_poison_vgg16)
print("")


while mlflow_run_id_is_not_known(response_gen_poison_vgg16):
    time.sleep(1)
    response_gen_poison_vgg16 = restapi_client.get_job_by_id(response_gen_poison_vgg16["jobId"])
    
# Wait for attack to finish.
while response_gen_poison_vgg16['status'] != "finished":
    time.sleep(1)
    response_gen_poison_vgg16 = restapi_client.get_job_by_id(
        response_gen_poison_vgg16["jobId"]
    )  

This will create a set of poisoned images that we can insert into the MNIST training set.

To see if the images can past inspection, we will now examine the image similarity metrics generated from our job and some sample images from the test set.


In [ ]:
mlflow_client = MlflowClient()
run_id = response_gen_poison_vgg16["mlflowRunId"]
view_run_vgg16 = mlflow_client.get_run(run_id)
pprint.pprint(view_run_vgg16.data.metrics)

In [ ]:
# We will also visually inspect the changes made to each poisoned image from its original counterpart:
load_images_from_job_run(run_id, "./poisoned_images_fruits")

def get_target_class_name(poison_dir):
    for item in os.listdir(poison_dir):
        if os.path.isdir(poison_dir +"/"+ item):
            return item
    return "None"

print(get_target_class_name("./poisoned_images_fruits/adv_poison_data"))

In [ ]:
from pathlib import Path

def check_image(image_path, descriptor):
    image = Image.open(image_path).resize((224,224))
    image = np.asarray(image)
        # plot the sample
    fig = plt.figure
    plt.title(descriptor)
    plt.imshow(image, cmap='gray')
    plt.show()
    return image


def compare_image_sets(original_model, src_image_dir, corrupt_image_dir):
    corrupt_image_list = [os.path.join(corrupt_image_dir, f) for f in os.listdir(corrupt_image_dir) if os.path.isfile(os.path.join(corrupt_image_dir, f))]
    
    corrupt_image = np.random.choice(corrupt_image_list,1)[0]
    
    original_image = os.path.join(src_image_dir, corrupt_image.split("poisoned_")[-1])
    
    target_image =  f"/nfs/1/datasets/Fruits360-Kaggle-2019/fruits-360/Test/{original_label}/{target_file}"
    
    image = check_image(target_image, "Target Test Image")
    print("Model prediction: " + str(np.argmax(original_model.predict(np.reshape(image, (1,224,224,3))), axis=1)[0]))

    print("Image: " + original_image)
    image = check_image(original_image, "Original Training Image")
    print("Model prediction: " + str(np.argmax(original_model.predict(np.reshape(image, (1,224,224,3))), axis=1)[0]))
    image = check_image(corrupt_image, "Corrupted Training Image")
    print("Model prediction: " + str(np.argmax(original_model.predict(np.reshape(image, (1,224,224,3))), axis=1)[0]))

fixed_target_label = target_label#.replace(" ", "\\ ")
src  = Path(f"/nfs/1/datasets/Fruits360-Kaggle-2019/fruits-360/Training/{fixed_target_label}")
corr_src = Path(f"./poisoned_images_fruits/adv_poison_data/{fixed_target_label}")

compare_image_sets(original_model,src, corr_src)
    

In [ ]:
# Deploy Poison to Dataset
response_deploy_poison_vgg16 = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_poison",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_gen_poison_vgg16['mlflowRunId']}",
            "-P data_dir=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training",
            "-P poison_deployment_method=replace",
            #"-P num_poisoned_images=100",
        ]
    ),
    queue="tensorflow_cpu",
    depends_on=response_gen_poison_vgg16["jobId"],
)


print("Poison deployment (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_poison_vgg16)
print("")

# Wait for Poison deployment to finish.
while mlflow_run_id_is_not_known(response_deploy_poison_vgg16):
    time.sleep(1)
    response_deploy_poison_vgg16 = restapi_client.get_job_by_id(response_deploy_poison_vgg16["jobId"])

In [ ]:
# Finally, train and retest poisoned dataset.

response_deploy_vgg16_poison_frogs_mnist_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join(
        [
            f"-P testing_dataset_run_id=None",
            f"-P training_dataset_run_id={response_deploy_poison_vgg16['mlflowRunId']}",
            "-P batch_size=256",
            "-P register_model=True",
            "-P model_architecture=vgg16",
            f"-P model_tag=poisoned_runID_{response_deploy_poison_vgg16['mlflowRunId']}",
            "-P epochs=30",
            "-P data_dir_train=/nfs/data/",
            "-P data_dir_test=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Test",
            "-P load_dataset_from_mlruns=True",

        ]
    ),
    
    queue="tensorflow_gpu",
    depends_on=response_deploy_poison_vgg16["jobId"],
)


print("Poison frogs adversarial training (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_vgg16_poison_frogs_mnist_adv_training)
print("")

# Wait for training to finish.
while mlflow_run_id_is_not_known(response_deploy_vgg16_poison_frogs_mnist_adv_training):
    time.sleep(1)
    response_deploy_vgg16_poison_frogs_mnist_adv_training = restapi_client.get_job_by_id(response_deploy_vgg16_poison_frogs_mnist_adv_training["jobId"])
    
while response_deploy_vgg16_poison_frogs_mnist_adv_training['status'] != "finished":
    time.sleep(1)
    response_deploy_vgg16_poison_frogs_mnist_adv_training = restapi_client.get_job_by_id(
        response_deploy_vgg16_poison_frogs_mnist_adv_training["jobId"]
    ) 

In [ ]:
# Finally, let's check if the attack worked, the target image should be reclassified as beloning to the new target class.

In [ ]:
corrupt_model = load_model_from_job_run(response_deploy_vgg16_poison_frogs_mnist_adv_training['mlflowRunId'], './test_dir')

In [ ]:
target_image = f"/nfs/1/datasets/Fruits360-Kaggle-2019/fruits-360/Test/{original_label}/{target_file}"

# load and show an image with Pillow
from PIL import Image
import tensorflow as tf
# Open the image form working directory
image = Image.open(target_image).resize((224,224))

# summarize some details about the image
print("Original Image label: " + str(original_label) )
#print("Predicted label (original model): " + str(np.argmax(original_model.predict(np.reshape(image, (1,224,224,3))), axis=1)[0]))
print("Predicted label (corrupted model): " + class_names[np.argmax(corrupt_model.predict(np.reshape(image, (1,224,224,3))), axis=1)[0]])
image = np.asarray(image)

# display the array of pixels as an image
import matplotlib.pyplot as plt
fig = plt.figure
plt.imshow(image)
plt.show()